# 1. Installing Python packages

In [1]:
!pip install tensorflow==2.12.0 tensorflowjs==2.0.0 opencv-python mediapipe scikit-learn

  Using cached prompt_toolkit-1.0.14-py3-none-any.whl.metadata (7.8 kB)
Using cached prompt_toolkit-1.0.14-py3-none-any.whl (248 kB)
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.43
    Uninstalling prompt-toolkit-3.0.43:
      Successfully uninstalled prompt-toolkit-3.0.43


DEPRECATION: tensorflowjs 2.0.0 has a non-standard dependency specifier tensorflow-cpu>=2.1.0<3. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflowjs or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.24.0 requires prompt-toolkit<3.1.0,>=3.0.41, but you have prompt-toolkit 1.0.14 which is incompatible.
jupyter-console 6.6.3 requires prompt-toolkit>=3.0.30, but you have prompt-toolkit 1.0.14 which is incompatible.


# 2. Importing libraries

In [2]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp
import tensorflowjs as tfjs
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D,Flatten, LSTM, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.regularizers import l2

# 3. Creating variables 

In [9]:
DATA_PATH = os.path.join('MP_Data')
signs = np.array([chr(i) for i in range(ord('A'), ord('Z')+1)] + ['next', 'space', 'backspace'])
mp_hands = mp.solutions.hands
modelH = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.4)
drawing_utils = mp.solutions.drawing_utils
drawing_styles = mp.solutions.drawing_styles
data_for_each_sign = 100
data_length = 45
sign_map = {sign: index for index, sign in enumerate(signs)}
labels_array = np.zeros((29*100, 29), dtype=int)
for i in range(29):
    labels_array[i*100:(i+1)*100, i] = 1

# 4. Defining usefull functions

In [4]:
def get_data(results):
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark
        arr = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks]).flatten()
    else:
        arr = np.zeros(63)
    return arr

In [5]:
def draw_landmarks(results, frame):
    if results.multi_hand_landmarks:
        for landmark in results.multi_hand_landmarks:
            drawing_utils.draw_landmarks(
                frame,  
                landmark,
                mp_hands.HAND_CONNECTIONS, 
                drawing_styles.get_default_hand_landmarks_style(),
                drawing_styles.get_default_hand_connections_style())
    return frame

# 5. Collecting data for training

In [6]:
for sign in signs: 
    for index in range(data_for_each_sign):
        try: 
            os.makedirs(os.path.join(DATA_PATH, sign, str(index)))
        except:
            pass

In [7]:
cap = cv2.VideoCapture(0)
for sign in signs:
    for index in range(data_for_each_sign):
        for data_index in range(data_length):
            ret, frame = cap.read()
            frame_flipped = cv2.flip(frame, 1)
            image = cv2.cvtColor(frame_flipped, cv2.COLOR_BGR2RGB)
            results = modelH.process(image) 

            drawn_frame = draw_landmarks(results, frame_flipped)
           
            if data_index != 0: 
                cv2.imshow('Frame', drawn_frame)
            else:
                cv2.rectangle(drawn_frame, (50,50), (150,150), (0, 0, 255), -1)
                cv2.putText(drawn_frame, sign, (75, 125), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 5)
                cv2.imshow('Frame', drawn_frame)
                cv2.waitKey(1000)
                
            np.save(os.path.join(DATA_PATH, sign, str(index), str(data_index)), get_data(results))
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
cap.release()
cv2.destroyAllWindows()

In [10]:
all_data = []
for sign in signs:
    for index in range(data_for_each_sign):
        video = []
        for data_index in range(data_length):
            loaded_data = np.load(os.path.join(DATA_PATH, sign, str(index), "{}.npy".format(data_index)))
            video.append(loaded_data)
        all_data.append(video)

In [11]:
X = np.array(all_data)
Y = labels_array
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

# 6. Creating and Training Model

In [12]:
model = Sequential()

# Input layer
input_layer = Input(shape=(45, 63), name='input_layer')
model.add(input_layer)

# Convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM layers
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(64))
model.add(Dropout(0.5))

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(29, activation='softmax')) # Assuming 29 classes

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, Y_train, epochs=200)

Epoch 1/200
73/73 [==============================] - 3s 11ms/step - loss: 3.3622 - accuracy: 0.0392
Epoch 2/200
73/73 [==============================] - 1s 11ms/step - loss: 2.9098 - accuracy: 0.1259
Epoch 3/200
73/73 [==============================] - 1s 11ms/step - loss: 2.3888 - accuracy: 0.2026
Epoch 4/200
73/73 [==============================] - 1s 11ms/step - loss: 2.0021 - accuracy: 0.3034
Epoch 5/200
73/73 [==============================] - 1s 11ms/step - loss: 1.6302 - accuracy: 0.3728
Epoch 6/200
73/73 [==============================] - 1s 11ms/step - loss: 1.6066 - accuracy: 0.3961
Epoch 7/200
73/73 [==============================] - 1s 11ms/step - loss: 1.3615 - accuracy: 0.4703
Epoch 8/200
73/73 [==============================] - 1s 12ms/step - loss: 1.2725 - accuracy: 0.5224
Epoch 9/200
73/73 [==============================] - 1s 12ms/step - loss: 1.0311 - accuracy: 0.5953
Epoch 10/200
73/73 [==============================] - 1s 11ms/step - loss: 1.0490 - accuracy: 0.5845

# 7. Saving model in H5 and TFJS format

In [15]:
model.save('model.h5')

In [16]:
model = tf.keras.models.load_model('model.h5')
tfjs.converters.save_keras_model(model, 'tfjs_model')